# Part 1: Conceptes Fonamentals d'Agents d'IA
## JCM Technologies - Departament de R+D

<table style="margin: 0; text-align: left; width:100%; border: 2px solid #2e86ab;">
    <tr>
        <td style="width: 150px; vertical-align: middle; padding: 20px;">
            <img src="https://raw.githubusercontent.com/googlefonts/noto-emoji/main/png/128/emoji_u1f9e0.png" width="120" style="display: block;" />
        </td>
        <td style="padding: 20px;">
            <h2 style="color:#2e86ab; margin: 0;">Objectius d'aprenentatge</h2>
            <ul style="color:#333; margin-top: 10px;">
                <li>Entendre la diferència entre <strong>workflows</strong> i <strong>agents</strong></li>
                <li>Crear el teu primer agent amb OpenAI</li>
                <li>Aplicar-ho a un cas real: analitzar un BOM (Bill of Materials)</li>
            </ul>
            <p style="color:#ff7800; margin: 10px 0 0 0;"><strong>Durada estimada:</strong> 50-60 minuts</p>
        </td>
    </tr>
</table>

---
## 📚 Continguts

1. **Recordatori ràpid:** Què són els LLMs? (5 min)
2. **Workflows vs Agents:** La diferència clau (15 min)
3. **Mans a l'obra:** El teu primer agent (15 min)
4. **Cas pràctic:** Agent analitzador de BOM (20 min)
5. **Reflexió final** (5 min)

---
## 1. Recordatori Ràpid: Què són els LLMs?

Ja coneixes ChatGPT i Copilot, així que aquest serà ràpid. Però anem a repassar els conceptes bàsics perquè són la base dels agents.

### 1.1 Els components bàsics

Un **Large Language Model (LLM)** és, essencialment:

```
Input (prompt) → [Model] → Output (resposta)
```

**Conceptes que ja coneixes:**
- **Prompt:** Les instruccions que dones al model
- **Tokens:** Les unitats que el model processa (aproximadament 4 caràcters per token)
- **Temperature:** Controla la "creativitat" (0 = determinista, 1 = creatiu)
- **Context window:** Quanta informació pot "recordar" el model

**El que potser no saps:** Quan cridem un LLM programàticament, no estem parlant amb ChatGPT directament. Estem cridant una **API** que ens dóna accés al model.

### 1.2 Exemple ràpid amb l'API d'OpenAI

Vegem com es fa una crida bàsica a l'API:

In [1]:
# Imports necessaris
from openai import OpenAI
from dotenv import load_dotenv
import os

# Carregar variables d'entorn (la nostra API key)
load_dotenv(override=True)

# Crear el client d'OpenAI
client = OpenAI()

print("✅ Client d'OpenAI inicialitzat correctament!")

✅ Client d'OpenAI inicialitzat correctament!


In [2]:
# Una crida bàsica a l'API
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Ets un assistent tècnic especialitzat en electrònica."},
        {"role": "user", "content": "Explica'm en una frase què és un díode."}
    ],
    temperature=0.7
)

print(response.choices[0].message.content)

Un díode és un dispositiu electrònic que permet el pas de corrent en una sola direcció i bloqueja el flux en l'altra, actuant com una vàlvula per a l'electricitat.


**Això és programació amb LLMs tradicional.** Però... **això NO és un agent encara!**

Abans de continuar, parem un moment per parlar de costos.

### 1.3 Costos i Rendiment dels Models

Una decisió tècnica important quan treballem amb agents és **quin model utilitzar**. No tots els models són iguals!

#### 💰 Per què importa el cost?

Cada crida a l'API té un cost. Si el teu agent fa:
- 100 crides al dia → Cost diari
- 1,000 crides al dia → Cost considerable
- 10,000 crides al dia → Has de vigilar molt el model!

**Missatge clau:** No sempre necessites el model més potent. Sovint un model més petit i ràpid és millor!

#### 📊 Comparativa de Models OpenAI (gener 2025)

| Model | Input ($/1M tokens) | Output ($/1M tokens) | Millor per a... |
|-------|---------------------|----------------------|------------------|
| **gpt-4o-mini** | $0.15 | $0.60 | Tasques senzilles, prototips, alt volum |
| **gpt-4o** | $2.50 | $10.00 | Tasques complexes, anàlisi avançada |
| **o1** | $15.00 | $60.00 | Raonament complex, matemàtiques |
| **o1-mini** | $3.00 | $12.00 | Raonament ràpid, cost moderat |

**Exemple pràctic:**
- Analitzar un BOM de 500 tokens amb GPT-4o-mini: ~$0.0003 (gairebé gratis!)
- La mateixa tasca amb GPT-4o: ~$0.005 (17x més car)
- Si fas 10,000 anàlisis/mes: $3 vs $50

**📍 Preus actualitzats:** https://openai.com/api/pricing/

#### 🏆 Rendiment vs Cost

No només és qüestió de cost. També has de considerar:
- **Qualitat:** Models més grans solen ser més precisos
- **Velocitat:** Models petits responen més ràpid
- **Capacitat:** Models grans gestionen tasques més complexes

**Recursos per comparar models:**
- **Chatbot Arena (LMSYS):** https://chat.lmsys.org/?leaderboard  
  *Comparativa de qualitat basada en preferències d'usuaris reals*
  
- **Artificial Analysis:** https://artificialanalysis.ai/  
  *La millor per veure cost vs qualitat vs velocitat*
  
- **OpenAI Evals:** https://github.com/openai/evals  
  *Benchmarks oficials d'OpenAI*

#### 🎯 Regla d'or per escollir model

1. **Comença sempre amb GPT-4.1-mini**  
   És el model més nou, ràpid, barat, i sorprenentment potent per la majoria de tasques

2. **Puja a GPT-4.1 (o GPT-5-mini) només si:**
   - La qualitat de GPT-4.1-mini no és suficient
   - La tasca requereix raonament més profund
   - Necessites seguir instruccions molt precises
   - El cost no és un problema

3. **Considera GPT-5 per:**
   - Tasques extremadament complexes
   - Quan necessites el màxim rendiment possible
   - Projectes crítics on la qualitat és essencial

4. **Utilitza o1/o1-mini per:**
   - Raonament complex específic (matemàtiques, lògica, ciència)
   - Quan necessites "pensar" més temps
   - Problemes que requereixen múltiples passos de raonament encadenats
   - Nota: o1 és més lent però més precís en raonament

**Per al nostre projecte d'escandalls:** GPT-4.1-mini serà més que suficient per a la majoria de tasques! És la millor opció qualitat/preu a finals de 2025.

💡 **Consell pràctic:** Fes sempre A/B testing. Prova la teva tasca amb diferents models i mesura:
- Qualitat del resultat
- Temps de resposta  
- Cost

Després decideix quin és el millor equilibri per al teu cas d'ús.

Informació ús i costos API: https://platform.openai.com/usage

---
## 2. Workflows vs Agents: La Diferència Clau

Aquesta és la secció més important conceptualment. Entendre aquesta diferència canviarà com penses sobre IA.

### 2.1 Què és un Workflow?

Un **workflow** (o patró agèntic) és una **seqüència predefinida** de crides a LLMs.

Tu, com a programador, decideixes:
- Quins LLMs es criden
- En quin ordre
- Amb quines condicions

**Exemples de workflows:**

#### 🔗 1. Prompt Chaining

**Descompondre una tasca en sub-tasques fixes**

```
INPUT → [LLM1] → [Gate] → [LLM2] → [LLM3] → OUTPUT
```

**Exemple:** 
1. LLM1 genera una pregunta
2. Gate decideix si continuar
3. LLM2 respon la pregunta  
4. LLM3 valida la resposta

**Característiques:**
- ✅ Seqüència lineal i predictible
- ✅ Fàcil de debugar
- ❌ No s'adapta a situacions noves

#### 🔀 2. Routing

**Dirigir l'input a un LLM especialitzat**

```
                    ↗ [LLM1]
INPUT → [Router] → → [LLM2] → OUTPUT
                    ↘ [LLM3]
```

**Exemple:**
- Router classifica la consulta (tècnica, comercial, suport)
- Envia a l'LLM expert corresponent

**Característiques:**
- ✅ Separació de concerns
- ✅ LLMs especialitzats per domini
- ❌ Les rutes són fixes

#### ⚡ 3. Parallelization

**Executar múltiples sub-tasques simultàniament**

```
                      → [LLM1] ↘
INPUT → [Coordinator] → [LLM2] → [Aggregator] → OUTPUT
                      → [LLM3] ↗
```

**Exemple:**
- Analitzar un document des de 3 perspectives diferents
- Agregar els resultats

**Característiques:**
- ✅ Més ràpid (paral·lel)
- ✅ Perspectives múltiples
- ❌ Complexitat en l'agregació

#### 🎭 4. Orchestrator-Worker

**Tasques complexes descompostes dinàmicament**

```
                        → [LLM1] ↘
INPUT → [Orchestrator] → [LLM2] → [Synthesizer] → OUTPUT
                        → [LLM3] ↗
```

**Exemple:**
- Orchestrator decideix quines sub-tasques cal fer
- Workers executen les tasques
- Synthesizer combina els resultats

**Característiques:**
- ✅ Descomposició dinàmica
- ✅ Escalable
- ❌ Encara és un workflow (no decideix en temps real)

#### 🔄 5. Evaluator-Optimizer

**Validar i millorar iterativament**

```
INPUT → [Generator] → Solució → [Evaluator] → ✅ OUTPUT
            ↑                         ↓
            └─────── Feedback ←───────┘
                    (si rebutjat)
```

**Exemple:**
- Generator crea una solució
- Evaluator la valida
- Si no és acceptable, torna a generar amb feedback

**Característiques:**
- ✅ Auto-millora
- ✅ Control de qualitat
- ❌ Pot ser lent (múltiples iteracions)

### 2.2 Aleshores... què és un Agent?

Un **agent** és diferent. L'agent:
- **Percep l'entorn** (llegeix informació)
- **Pren decisions** sobre què fer
- **Actua** (executa accions)
- **Rep feedback** de l'entorn
- **Itera** fins assolir l'objectiu

**Esquema d'un agent:**

```
                    ← Action ──┐
HUMAN → [LLM Agent]            [ENVIRONMENT]
          ↓    ↑               ↓
        [STOP] └── Feedback ───┘
```

**Característiques clau:**
- ✅ Pren decisions autònomes
- ✅ S'adapta segons el feedback
- ✅ Pot usar eines (tools)
- ⚠️ Més difícil de controlar
- ⚠️ Pot ser impredictible

### 2.3 Comparació directa

| Aspecte | Workflow | Agent |
|---------|----------|-------|
| **Control** | Tu decideixes el flux | L'agent decideix |
| **Adaptabilitat** | Flux fix predefinit | S'adapta dinàmicament |
| **Predictibilitat** | Molt predictible | Menys predictible |
| **Complexitat** | Més simple | Més complex |
| **Autonomia** | Zero autonomia | Alta autonomia |
| **Cas d'ús** | Processos coneguts | Situacions obertes |

**Regla d'or:**
- Si **saps exactament** els passos → Workflow
- Si l'agent ha de **decidir** què fer → Agent

### 💡 Reflexió

Pensa en el cas d'ús de valoració d'escandalls que farem més endavant:

**Pregunta:** Creus que hauria de ser un workflow o un agent? Per què?

*Pensa-ho abans de continuar...*

---
## 3. Mans a l'Obra: El Teu Primer Agent

Ara que entens la teoria, anem a crear el nostre primer agent amb l'**OpenAI Agents SDK**.

### 3.1 Per què utilitzar l'OpenAI Agents SDK?

Podríem construir agents manualment amb l'API d'OpenAI, però l'SDK ens dóna:

✅ Gestió automàtica del **cicle agent** (percepció → decisió → acció)
✅ **Tools** (funcions que l'agent pot cridar) integrades
✅ **Traces** per debugar el comportament de l'agent
✅ **Handoffs** entre agents
✅ Gestió de **context** i **memòria**

**En resum:** L'SDK fa el heavy lifting per nosaltres.

### 3.2 Anatomia d'un Agent

Un agent té aquests components bàsics:

```python
Agent(
    name="Nom de l'agent",           # Identificador
    instructions="Què ha de fer",    # El "prompt" de l'agent
    model="gpt-4o-mini",             # El LLM a utilitzar
    tools=[...]                       # Eines que pot utilitzar (opcional)
)
```

Les **instructions** són crítiques: defineixen el "caràcter" i comportament de l'agent.

### 3.3 El nostre primer agent: l'Analitzador Tècnic

Crearem un agent simple que entén d'electrònica.

In [3]:
from agents import Agent, Runner, trace
from IPython.display import display, Markdown

# Definim les instruccions de l'agent
instructions = """
Ets un enginyer electrònic expert que treballa a JCM Technologies.

El teu rol és analitzar components electrònics i proporcionar informació tècnica clara i precisa.

Quan responguis:
- Sigues concís però complet
- Utilitza terminologia tècnica adequada
- Si no estàs segur d'alguna cosa, digues-ho
"""

# Crear l'agent
agent_tecnic = Agent(
    name="Analitzador Tècnic",
    instructions=instructions,
    model="gpt-4o-mini"
)

print("✅ Agent creat correctament!")

✅ Agent creat correctament!


### 3.4 Executar l'agent

#### ⚡ Abans d'executar l'agent: què és `async/await`?

Quan executem l'agent, utilitzarem aquesta sintaxis:
```python
result = await Runner.run(agent, consulta)
```

**Què significa `await`?**

`async/await` és **programació asíncrona**. Permet que el teu programa no es quedi "bloquejat" esperant una resposta lenta (com una crida a l'API d'OpenAI).

**Analogia:** És com demanar un cafè ☕
- **Sync (bloquejant):** T'esperes al taulell mirant fixament la màquina fins que estigui llest (perdent temps!)
- **Async (no bloquejant):** Demanes el cafè i fas altres coses mentre el preparen

**Per què ho necessitem amb agents?**
- Les crides a l'API poden trigar segons
- Els agents poden fer múltiples crides
- Amb async, podem fer altres coses mentre esperem (o fins i tot executar múltiples agents en paral·lel!)

**Regles pràctiques:**
✅ Sempre posa `await` abans de `Runner.run()`  
✅ Jupyter i Cursor suporten async automàticament  
❌ Si oblides el `await`, veuràs un error sobre "coroutine"

**Vols saber més?** https://realpython.com/async-io-python/

Ara sí, executem el nostre primer agent! 👇


In [4]:
# Definim la consulta
consulta = "Què és un relé i per què s'utilitza?"

# Executem l'agent amb trace per poder veure què fa
with trace("Consulta tècnica"):
    result = await Runner.run(agent_tecnic, consulta)

# Mostrem el resultat
display(Markdown(result.final_output))

Un relé és un dispositiu electromecànic utilitzat per obrir i tancar circuits elèctrics mitjançant un senyal elèctric. Consisteix en una bobina, un contacte (normalment constituït per un conjunt de peces metàl·liques), i un nucli de ferro. Quan passa corrent a través de la bobina, es genera un camp magnètic que atrau el nucli, provocant l'obertura o tancament dels contactes.

Els relés s'utilitzen per:

1. **Controlar càrregues elèctriques**: Permeten encendre o apagar dispositius d'alta potència amb un senyal de baixa potència.
   
2. **Aïllament galvànic**: Ofereixen separació entre el circuit de control i el circuit de potència, protegint així components sensibles.

3. **Automatització**: Són essencials en sistemes de control automàtic i en aplicacions domèstiques i industrials, com en sistemes de seguretat, controls de motors i màquines.

4. **Multiplexió de circuits**: Permeten controlar diversos circuits amb un sol senyal d'entrada.

En resum, els relés són components versàtils que faciliten el control de circuits elèctrics de manera fiable i segura.

### 🔍 Observa el trace!

Ves a https://platform.openai.com/traces i busca el trace "Consulta tècnica".

Veuràs:
- L'input que hem donat
- Les instruccions de l'agent
- La resposta del model
- El temps que ha trigat
- Els tokens utilitzats

**Això és extremadament útil per debugar agents!**

### 💡 Reflexió

Prova a fer diverses consultes diferents a l'agent:
- Una consulta tècnica sobre un component
- Una pregunta fora del seu domini (ex: receptes de cuina)
- Una pregunta ambigua

**Pregunta:** Com es comporta l'agent en cada cas? Es manté "en el personatge"?

In [ ]:
# Prova les teves pròpies consultes aquí
consulta_personalitzada = "Passa'm una recepta de pollastre estofat"  # Escriu la teva consulta

# Descomenta per executar:
# with trace("Consulta personalitzada"):
#    result = await Runner.run(agent_tecnic, consulta_personalitzada)
# display(Markdown(result.final_output))

---
## 4. Cas Pràctic: Agent Analitzador de BOM

Ara aplicarem el que hem après a un cas real: analitzar un Bill of Materials (BOM).

### 4.1 Què és un BOM?

Un **Bill of Materials** (Escandall) és una llista estructurada de tots els components necessaris per fabricar un producte.

**Exemple simplificat:**

| Part Number | Description | Quantity | Unit |
|-------------|-------------|----------|------|
| RES-001 | Resistència 10kΩ | 5 | pcs |
| CAP-002 | Condensador 100nF | 3 | pcs |
| LED-003 | LED vermell 5mm | 2 | pcs |
| IC-004 | Microcontrolador ATmega328 | 1 | pcs |

### 4.2 Objectiu: Agent que analitza un BOM

Volem un agent que pugui:
1. Llegir un BOM (text o dades estructurades)
2. Identificar els components
3. Agrupar-los per categoria
4. Detectar possibles problemes o inconsistències

Comencem amb un BOM simplificat en format text:

In [5]:
# BOM simplificat per la demo
bom_simple = """
BILL OF MATERIALS - Control Board v2.1

1. RES-10K - Resistència 10kΩ 1/4W - Qty: 5
2. RES-1K - Resistència 1kΩ 1/4W - Qty: 3  
3. CAP-100N - Condensador ceràmic 100nF - Qty: 4
4. CAP-10U - Condensador electrolític 10µF - Qty: 2
5. LED-RED - LED vermell 5mm - Qty: 2
6. LED-GREEN - LED verd 5mm - Qty: 1
7. RELAY-5V - Relé 5V 10A SPDT - Qty: 3
8. IC-ATMEGA - Microcontrolador ATmega328P - Qty: 1
9. CONN-RJ45 - Connector RJ45 - Qty: 1
10. PCB - Placa de circuit imprès - Qty: 1
"""

print(bom_simple)


BILL OF MATERIALS - Control Board v2.1

1. RES-10K - Resistència 10kΩ 1/4W - Qty: 5
2. RES-1K - Resistència 1kΩ 1/4W - Qty: 3  
3. CAP-100N - Condensador ceràmic 100nF - Qty: 4
4. CAP-10U - Condensador electrolític 10µF - Qty: 2
5. LED-RED - LED vermell 5mm - Qty: 2
6. LED-GREEN - LED verd 5mm - Qty: 1
7. RELAY-5V - Relé 5V 10A SPDT - Qty: 3
8. IC-ATMEGA - Microcontrolador ATmega328P - Qty: 1
9. CONN-RJ45 - Connector RJ45 - Qty: 1
10. PCB - Placa de circuit imprès - Qty: 1



### 4.3 Crear l'agent analitzador de BOM

In [6]:
instructions_bom = """
Ets un enginyer electrònic especialitzat en analitzar Bills of Materials (BOMs).

Quan rebs un BOM, has de:
1. Identificar tots els components llistats
2. Agrupar-los per categories (resistències, condensadors, semiconductors, connectors, etc.)
3. Comptar el total de components per categoria
4. Identificar possibles problemes o inconsistències

Presenta la informació de manera clara i estructurada.
"""

agent_bom = Agent(
    name="Analitzador BOM",
    instructions=instructions_bom,
    model="gpt-4o-mini"
)

print("✅ Agent Analitzador BOM creat!")

✅ Agent Analitzador BOM creat!


In [7]:
# Analitzem el BOM
consulta_bom = f"""
Analitza el següent BOM i proporciona un resum estructurat:

{bom_simple}
"""

with trace("Anàlisi BOM"):
    result = await Runner.run(agent_bom, consulta_bom)

display(Markdown(result.final_output))

### Resum de l'Anàlisi del BOM - Control Board v2.1

#### 1. Components Identificats
- **Resistències:**
  - RES-10K - Resistència 10kΩ 1/4W - Qty: 5
  - RES-1K - Resistència 1kΩ 1/4W - Qty: 3

- **Condensadors:**
  - CAP-100N - Condensador ceràmic 100nF - Qty: 4
  - CAP-10U - Condensador electrolític 10µF - Qty: 2

- **LLums:**
  - LED-RED - LED vermell 5mm - Qty: 2
  - LED-GREEN - LED verd 5mm - Qty: 1

- **Relés:**
  - RELAY-5V - Relé 5V 10A SPDT - Qty: 3

- **Microcontroladors:**
  - IC-ATMEGA - Microcontrolador ATmega328P - Qty: 1

- **Connectors:**
  - CONN-RJ45 - Connector RJ45 - Qty: 1

- **Placa de Circuit Imprès:**
  - PCB - Placa de circuit imprès - Qty: 1

#### 2. Agrupació de Components per Categories
- **Resistències:**
  - 2 tipus (RES-10K, RES-1K)
  - Total: 8

- **Condensadors:**
  - 2 tipus (CAP-100N, CAP-10U)
  - Total: 6

- **LLums:**
  - 2 tipus (LED-RED, LED-GREEN)
  - Total: 3

- **Relés:**
  - 1 tipus (RELAY-5V)
  - Total: 3

- **Microcontroladors:**
  - 1 tipus (IC-ATMEGA)
  - Total: 1

- **Connectors:**
  - 1 tipus (CONN-RJ45)
  - Total: 1

- **Placa de Circuit Imprès:**
  - 1 tipus (PCB)
  - Total: 1

#### 3. Comptatge Total de Components per Categoria
- **Resistències:** 8
- **Condensadors:** 6
- **LLums:** 3
- **Relés:** 3
- **Microcontroladors:** 1
- **Connectors:** 1
- **Placa de Circuit Imprès:** 1

#### 4. Problemes o Inconsistències Identificades
- No hi ha discrepàncies evidents en el BOM; tots els components semblen estar ben classificats i llistats amb la seva quantitat especificada.
- Potser es podria revisar la compatibilitat entre el relé i el microcontrolador, depenent de les condicions de disparo i càrrega.

### Conclusió
El BOM presentat per la "Control Board v2.1" està ben estructurat i no presenta inconsistències immediates. Es recomana verificar la compatibilitat dels components crítics abans de la seva implementació.

### 4.4 Millorant l'agent: Structured Outputs

El problema de l'agent anterior és que la sortida és text lliure. Seria millor tenir una sortida estructurada que puguem processar programàticament.

Pydantic ens permet definir **esquemes de sortida** que l'agent seguirà automàticament.

In [8]:
from pydantic import BaseModel, ConfigDict
from typing import List, Dict

class Component(BaseModel):
    """Un component del BOM"""
    model_config = ConfigDict(extra="forbid")
    part_number: str
    description: str
    quantity: int
    category: str

class BOMAnalysis(BaseModel):
    """Anàlisi estructurada d'un BOM"""
    model_config = ConfigDict(extra="forbid")
    components: List[Component]
    total_components: int
    categories_summary: Dict[str, int]  # ← Correcció aquí!
    observations: List[str]

print("✅ Esquema de dades definit!")

✅ Esquema de dades definit!


In [9]:
from agents import AgentOutputSchema
# Ara creem un agent que retorna dades estructurades
instructions_bom_structured = """
Ets un enginyer electrònic especialitzat en analitzar Bills of Materials (BOMs).

Quan rebs un BOM en format text, has de:
1. Extreure cada component amb el seu part number, descripció i quantitat
2. Classificar cada component en una categoria apropiada
3. Comptar el total de components i per categoria
4. Identificar possibles problemes o inconsistències

Retorna les dades en el format estructurat especificat.
"""

agent_bom_structured = Agent(
    name="Analitzador BOM Estructurat",
    instructions=instructions_bom_structured,
    model="gpt-4o-mini",
    output_type=AgentOutputSchema(BOMAnalysis, strict_json_schema=False)  # strict_json_schema=False no hi hauria d'anar, és per un problema de verions de Pydantic
)

print("✅ Agent amb sortida estructurada creat!")

✅ Agent amb sortida estructurada creat!


In [10]:
# Executem l'agent amb trace
with trace("Anàlisi BOM Estructurat"):
    result = await Runner.run(
        agent_bom_structured, 
        f"Analitza aquest BOM:\n\n{bom_simple}"
    )

# El resultat ja és un objecte BOMAnalysis validat per Pydantic!
bom_analysis = result.final_output

print("✅ Anàlisi completada!\n")
print(f"Total de components: {bom_analysis.total_components}")
print(f"\nComponents per categoria:")
for categoria, count in bom_analysis.categories_summary.items():
    print(f"  - {categoria}: {count}")

if bom_analysis.observations:
    print(f"\nObservacions:")
    for obs in bom_analysis.observations:
        print(f"  - {obs}")

✅ Anàlisi completada!

Total de components: 10

Components per categoria:
  - Resistències: 2
  - Condensadors: 2
  - LEDs: 2
  - Relés: 1
  - ICs: 1
  - Connectors: 1
  - Plates: 1

Observacions:
  - No es van identificar inconsistències o problemes en el BOM.


**💡 Diferència clau amb la secció anterior:**

A la secció 4.3, l'agent retornava text lliure (markdown).  
Ara, amb `output_type=BOMAnalysis`, l'agent **garanteix** que la sortida segueix l'estructura Pydantic.

Això és molt potent perquè:
- L'agent no pot "inventar-se" camps nous
- Tots els camps obligatoris estaran presents
- Els tipus seran correctes
- La validació és automàtica

**És com posar "barreres de seguretat" a l'agent!** 🛡️

### 4.5 Visualitzar els components

Ara que tenim dades estructurades, podem visualitzar-les fàcilment:

In [11]:
import pandas as pd

# Convertir a DataFrame per visualitzar millor
df_components = pd.DataFrame([c.model_dump() for c in bom_analysis.components])
df_components

,part_number,description,quantity,category
0,RES-10K,Resistència 10kΩ 1/4W,5,Resistències
1,RES-1K,Resistència 1kΩ 1/4W,3,Resistències
2,CAP-100N,Condensador ceràmic 100nF,4,Condensadors
3,CAP-10U,Condensador electrolític 10µF,2,Condensadors
4,LED-RED,LED vermell 5mm,2,LEDs
5,LED-GREEN,LED verd 5mm,1,LEDs
6,RELAY-5V,Relé 5V 10A SPDT,3,Relés
7,IC-ATMEGA,Microcontrolador ATmega328P,1,ICs
8,CONN-RJ45,Connector RJ45,1,Connectors
9,PCB,Placa de circuit imprès,1,Plates


In [12]:
# Resum per categories
df_categories = pd.DataFrame([
    {"Categoria": cat, "Quantitat": count} 
    for cat, count in bom_analysis.categories_summary.items()
])
df_categories

,Categoria,Quantitat
0,Resistències,2
1,Condensadors,2
2,LEDs,2
3,Relés,1
4,ICs,1
5,Connectors,1
6,Plates,1


### 💡 Reflexió Important

**Pregunta:** L'agent que acabem de crear, és realment un "agent" segons la definició que hem vist?

Pensa en:
- Pren decisions autònomes?
- Interactua amb l'entorn?
- Utilitza tools?
- Rep feedback i s'adapta?

**Spoiler:** En realitat, això encara és més proper a un workflow (una crida única al LLM) que a un veritable agent autònom.

**A la Part 2** veurem com convertir això en un agent real afegint **tools** (eines) que li permetin llegir fitxers, accedir a bases de dades, etc.

---
## 5. Reflexió Final

### Què hem après?

1. ✅ **Diferència entre workflows i agents**
   - Workflows: flux predefinit i controlat
   - Agents: autònoms, adaptatiu, amb feedback

2. ✅ **Patrons agèntics comuns**
   - Prompt Chaining, Routing, Parallelization
   - Orchestrator-Worker, Evaluator-Optimizer

3. ✅ **Crear agents amb OpenAI Agents SDK**
   - Agent, Runner, trace
   - Instructions, model, tools

4. ✅ **Cas pràctic: Analitzador de BOM**
   - Sortida estructurada amb Pydantic
   - Processament de dades reals

### Pròxims passos

A la **Part 2: Tools i Function Calling** veurem:

- Com donar "superpoders" als agents amb **tools**
- Crear tools personalitzades per:
  - Llegir fitxers Excel (BOMs reals)
  - Enviar emails (via Gmail API)
  - Accedir a bases de dades
- Convertir el nostre analitzador de BOM en un veritable agent autònom

A la **Part 3: Cas Pràctic Complet** integrarem tot per crear un sistema multi-agent per valoració d'escandalls.

---
<table style="margin: 20px 0; text-align: left; width:100%; border: 2px solid #27ae60;">
    <tr>
        <td style="width: 150px; vertical-align: middle; padding: 20px;">
            <img src="https://raw.githubusercontent.com/googlefonts/noto-emoji/main/png/128/emoji_u1f389.png" width="120" style="display: block;" />
        </td>
        <td style="padding: 20px;">
            <h2 style="color:#27ae60; margin: 0;">Enhorabona!</h2>
            <p style="color:#333; margin: 10px 0;">Has completat la Part 1 dels fonaments d'agents d'IA.</p>
            <p style="color:#ff7800; margin: 10px 0 0 0;"><strong>Proper pas:</strong> Part 2 - Tools i Function Calling</p>
        </td>
    </tr>
</table>